In [25]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV

In [26]:
import matplotlib.pyplot as plt

In [27]:
def convert_monetary(value):
    if not value or value.strip() == '':
        return np.nan

    if value == '€0':
        return 0

    multiplier = 1
    if 'M' in value:
        multiplier = 1e6
    elif 'B' in value:
        multiplier = 1e9

    numeric_value = float(value[1:-1]) * multiplier

    return numeric_value

In [28]:
def label_result(home_goals, away_goals):
    if home_goals > away_goals:
        return 'Home Win'
    elif home_goals < away_goals:
        return 'Away Win'
    else:
        return 'Draw'

In [29]:
df = pd.read_csv('../data/all_games_stats_with_team_ratings.csv')

In [30]:
df['Result'] = df.apply(lambda row: label_result(row['Home_Goals'], row['Away_Goals']), axis=1)

In [31]:
df.loc[:, ['Season', 'Home_Team_Name', 'Away_Team_Name', 'Home_Goals', 'Away_Goals', 'Result']].head()

,Season,Home_Team_Name,Away_Team_Name,Home_Goals,Away_Goals,Result
0,2015,Aston Villa,Everton,1,3,Away Win
1,2015,AFC Bournemouth,Southampton,2,0,Home Win
2,2015,Leicester City,West Bromwich Albion,2,2,Draw
3,2015,Norwich City,Chelsea,1,2,Away Win
4,2015,Sunderland,Crystal Palace,2,2,Draw


In [32]:
df.drop(['Season', 'Home_Goals', 'Away_Goals', 'Home_shots_on_goal', 'Home_shots_outsidebox', 'Home_shots_off_goal', 'Home_total_shots', 'Home_blocked_shots', 'Home_shots_insidebox', 'Home_corner_kicks', 'Home_ball_possession', 'Home_yellow_cards', 'Home_goalkeeper_saves', 'Home_total_passes', 'Home_passes_accurate', 'Home_passes_%', 'Away_shots_on_goal', 'Away_shots_off_goal', 'Away_total_shots', 'Away_blocked_shots', 'Away_shots_insidebox', 'Away_shots_outsidebox', 'Away_corner_kicks', 'Away_offsides', 'Away_ball_possession', 'Away_yellow_cards', 'Away_goalkeeper_saves', 'Away_total_passes', 'Away_passes_accurate', 'Away_passes_%', 'Home_offsides', 'Away_red_cards', 'Home_red_cards', 'Home_fouls', 'Away_fouls', 'Home_expected_goals', 'Away_expected_goals'], axis=1, inplace=True)

In [33]:
df.columns

Index(['Match_Date', 'Home_Team_Name', 'Away_Team_Name', 'Home_Overall',
       'Home_Attack', 'Home_Midfield', 'Home_Defence', 'Home_Transfer budget',
       'Home_Club worth', 'Home_Speed', 'Home_Dribbling', 'Home_Passing',
       'Home_Positioning', 'Home_Crossing', 'Home_Shooting', 'Home_Aggression',
       'Home_Pressure', 'Home_Team width', 'Home_Defender line',
       'Home_Domestic prestige', 'Home_International prestige', 'Home_Players',
       'Home_Starting XI average age', 'Home_Whole team average age',
       'Away_Overall', 'Away_Attack', 'Away_Midfield', 'Away_Defence',
       'Away_Transfer budget', 'Away_Club worth', 'Away_Speed',
       'Away_Dribbling', 'Away_Passing', 'Away_Positioning', 'Away_Crossing',
       'Away_Shooting', 'Away_Aggression', 'Away_Pressure', 'Away_Team width',
       'Away_Defender line', 'Away_Domestic prestige',
       'Away_International prestige', 'Away_Players',
       'Away_Starting XI average age', 'Away_Whole team average age',
       '

In [34]:
print(df['Match_Date'].unique())

[ 1032016  1052016  1112015  2012016  2022016  2032016  2042016  2052016
  2112015  3012016  3022016  3042016  3102015  4102015  5032016  5122015
  6022016  6032016  6122015  7022016  7052016  7112015  7122015  8052016
  8082015  8112015  9042016  9082015 10042016 10052016 10082015 11052016
 12012016 12032016 12092015 12122015 13012016 13022016 13032016 13042016
 13092015 13122015 14022016 14032016 14082015 14092015 14122015 15052016
 15082015 16012016 16042016 16082015 17012016 17042016 17052016 17082015
 17102015 18012016 18042016 18102015 19032016 19042016 19092015 19102015
 19122015 20032016 20042016 20092015 20122015 21042016 21112015 21122015
 22082015 22112015 23012016 23042016 23082015 23112015 24012016 24042016
 24082015 24102015 25042016 25102015 26092015 26122015 27022016 27092015
 28022016 28092015 28112015 28122015 29082015 29112015 29122015 30042016
 30082015 30122015 31102015  1012017  1022017  1042017  1052017  1102016
  2012017  2042017  2102016  3012017  3122016  4012

In [35]:
df['Match_Date'] = df['Match_Date'].astype(str)

df['Month'] = df['Match_Date'].str.extract(r'(\d{1,2})(?=\d{4}$)')

df['Month'] = df['Month'].str.zfill(2)

In [36]:
df['Month'] = pd.to_numeric(df['Month'])

In [37]:
df.drop(['Match_Date'], axis=1, inplace=True)

In [38]:
print(df['Month'].unique())

[ 3  5 11  1  2  4 10 12  8  9  7  6]


In [39]:
df['Home_Transfer budget'].unique()

df['Home_Club worth'].unique()

array(['€0', '€45M', '€525.6M', '€900M', '€780M', '€2.4B', '€2B', '€400M',
       '€2.2B', '€800M', '€560M', '€1.3B', '€650M', '€406.6M', '€3.6B',
       '€1.9B', '€1.8B', '€555.6M', '€14M', '€26M', '€1.4B', '€435M',
       '€2.5B', '€660M', '€1.1B', '€3B', '€525M', '€390M', '€615M',
       '€120M', '€540M', '€2.3B', '€635M', '€80M', '€440M', '€575M',
       '€2.8B', '€983M', '€90M', '€981.5M', '€640M', '€979M', '€3.5B',
       '€425M', '€15M', '€50M', '€551.3M', '€3.4B', '€383.3M', '€418.5M',
       '€401.6M', '€462M', '€1.7B', '€200M', '€332.5M', '€388.5M',
       '€2.7B', '€180M', '€378.5M', '€2.1B', '€441M', '€430.5M',
       '€682.5M', '€415M', '€533.3M', '€568.5M', '€455.5M', '€100M',
       '€3.3B', '€505M', '€552.5M', '€336M', '€65M', '€110M', '€410.5M',
       '€562.5M', '€565M', '€185.5M', '€411.5M', '€572.5M', '€420.5M',
       '€416.5M', '€107.5M', '€172.5M', '€220.5M', '€178.5M', '€2.9B',
       '€4.6B', '€169.5M', '€228.5M', '€741.5M', '€684.5M', '€730.5M',
       '€2.6B'

In [40]:
df['Home_Transfer budget'] = df['Home_Transfer budget'].apply(convert_monetary)
df['Home_Club worth'] = df['Home_Club worth'].apply(convert_monetary)

In [41]:
df.head()

,Home_Team_Name,Away_Team_Name,Home_Overall,Home_Attack,Home_Midfield,Home_Defence,Home_Transfer budget,Home_Club worth,Home_Speed,Home_Dribbling,...,Away_Pressure,Away_Team width,Away_Defender line,Away_Domestic prestige,Away_International prestige,Away_Players,Away_Starting XI average age,Away_Whole team average age,Result,Month
0,Aston Villa,Everton,75,72,74,74,11500000.0,0.0,Balanced,Normal,...,Medium,Normal,Cover,14,16,33,26.36,24.97,Away Win,3
1,AFC Bournemouth,Southampton,69,69,69,68,3000000.0,0.0,Balanced,Normal,...,Medium,Normal,Cover,11,13,33,26.18,22.82,Home Win,3
2,Leicester City,West Bromwich Albion,72,71,72,68,9500000.0,0.0,Balanced,Normal,...,Medium,Normal,Cover,11,14,33,27.91,24.36,Draw,3
3,Norwich City,Chelsea,72,73,72,71,6500000.0,0.0,Balanced,Normal,...,Medium,Normal,Cover,20,20,33,25.82,22.48,Away Win,3
4,Sunderland,Crystal Palace,74,76,73,72,16000000.0,0.0,Balanced,Normal,...,Medium,Normal,Cover,10,11,33,27.36,25.45,Draw,3


In [42]:
columns_to_label_encode = ['Home_Team_Name', 'Away_Team_Name', 'Home_Speed', 'Home_Dribbling', 'Home_Passing', 'Home_Positioning', 'Home_Crossing', 'Home_Shooting', 'Home_Aggression', 'Home_Pressure', 'Home_Team width', 'Home_Defender line', 'Away_Transfer budget', 'Away_Club worth', 'Away_Speed', 'Away_Dribbling', 'Away_Passing', 'Away_Positioning', 'Away_Crossing', 'Away_Shooting', 'Away_Aggression', 'Away_Pressure', 'Away_Team width', 'Away_Defender line']
label_encoder = LabelEncoder()
for column in columns_to_label_encode:
    df[column] = label_encoder.fit_transform(df[column].astype(str))

In [43]:
X = df.drop(['Result'], axis=1)
y_labels = df['Result']

In [44]:
class_mapping = {'Away Win': 0, 'Draw': 1, 'Home Win': 2}
y = y_labels.map(class_mapping)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
model = xgb.XGBClassifier(objective='multi:softmax', num_class=3)

model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=3,
              num_parallel_tree=None, ...)

In [47]:
param_grid = {
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [50, 100],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 5],
    'gamma': [0, 0.2],
}

In [48]:
grid_search = GridSearchCV(model, param_grid, scoring='accuracy', cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: {:.2f}%".format(grid_search.best_score_ * 100))

Best Parameters:  {'colsample_bytree': 1.0, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 50, 'subsample': 1.0}
Best Accuracy: 52.77%
